In [367]:
import pandas as pd
import plotly.express as px
import re
import datetime

from Project.Database import Db

In [368]:
year1_hourly, meta = Db.load_data(hourly=True, meta=True, year=1)

In [369]:
consumption_condition = (lambda self: (
        (self["Units"] == "W") &
        self["Description"].str.contains("power consumption" or "used") &
        ~(self["Unnamed: 0"] == "Elec_PowerMicrowave") &
        ~(self["Unnamed: 0"] == "Elec_PowerRefrigerator") &
        ~(self["Unnamed: 0"] == "Elec_PowerClothesWasher") &
        ~self["Unnamed: 0"].str.contains("Elec_[\w]+HVAC") &
        (~self["Unnamed: 0"].str.contains("DHW_") |
         self["Unnamed: 0"].str.contains("DHW_[\w]+Total")) &
        (~self["Unnamed: 0"].str.contains("Load_") |
         ~self["Unnamed: 0"].str.contains("Load_Microwave") &
         self["Unnamed: 0"].str.contains("Load_[\w]+Standby"))))


consumption = meta.loc[consumption_condition]["Unnamed: 0"].tolist()

production = meta.loc[(meta['Parameter'] == "Power_Electrical") & (
    meta["Description"].str.contains("produced"))]["Unnamed: 0"].tolist()

In [370]:
year1_consumption_production = year1_hourly[["Timestamp"]+consumption+production].copy()

In [371]:
year1_consumption_production["Mean_consumption"] = year1_consumption_production[consumption].sum(1)
year1_consumption_production["Mean_production"] = year1_consumption_production[production].sum(1)

In [372]:
agg_consum_prod = year1_consumption_production[["Timestamp","Mean_consumption","Mean_production"]].copy()

In [382]:
#Forces Timestamp to the type of datetime, to extract the hour of Timestamp.
agg_consum_prod['Timestamp'] = pd.to_datetime(agg_consum_prod['Timestamp'], errors="coerce", utc=True, format="%Y-%m-%d %H:%M:%S%z")

#Extracts hour of Timestamp.
agg_consum_prod["MonthOfTimestamp"] = agg_consum_prod.Timestamp.dt.month
agg_consum_prod["HourOfTimestamp"] = agg_consum_prod.Timestamp.dt.hour
agg_consum_prod

,Timestamp,Mean_consumption,Mean_production,MonthOfTimestamp,HourOfTimestamp,Proportion_of_consumption_production
0,2013-07-01 04:00:00+00:00,1006.317361,18.506595,7,4,0.018390
1,2013-07-01 05:00:00+00:00,629.003143,18.494896,7,5,0.029404
2,2013-07-01 06:00:00+00:00,976.929283,18.376221,7,6,0.018810
3,2013-07-01 07:00:00+00:00,676.805590,18.470420,7,7,0.027291
4,2013-07-01 08:00:00+00:00,707.015157,18.338783,7,8,0.025938
...,...,...,...,...,...,...
8756,2014-07-01 00:00:00+00:00,1702.495519,42.879678,7,0,0.025186
8757,2014-07-01 01:00:00+00:00,2516.681689,18.101134,7,1,0.007192
8758,2014-07-01 02:00:00+00:00,1367.653306,18.409789,7,2,0.013461
8759,2014-07-01 03:00:00+00:00,2501.732181,17.882693,7,3,0.007148


In [383]:
agg_consum_prod["Proportion_of_consumption_production"] = agg_consum_prod["Mean_production"] - agg_consum_prod["Mean_consumption"]
hourON = agg_consum_prod.groupby(["MonthOfTimestamp", "HourOfTimestamp"]).mean()
hourON.reset_index(inplace=True)


hourON

,MonthOfTimestamp,HourOfTimestamp,Mean_consumption,Mean_production,Proportion_of_consumption_production
0,1,0,2985.868974,18.411803,-2967.457171
1,1,1,2821.263247,18.540778,-2802.722469
2,1,2,3585.170426,18.688463,-3566.481963
3,1,3,3247.913173,18.735128,-3229.178045
4,1,4,2600.602557,18.789207,-2581.813350
...,...,...,...,...,...
283,12,19,1243.010613,1958.302925,715.292311
284,12,20,1671.809133,1012.433992,-659.375141
285,12,21,1102.800315,128.462439,-974.337876
286,12,22,1126.469904,17.451051,-1109.018853


In [388]:
import plotly.graph_objects as go

#Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=hourON["Proportion_of_consumption_production"],
    x=hourON["HourOfTimestamp"],
    y=hourON["MonthOfTimestamp"], colorscale="RdBu", zmid=0
)
)

fig.update_layout(margin=dict(
    l=30,
    r=30,
    b=30,
    t=50,
), xaxis_nticks=30,
    yaxis_nticks=20
    # annotations=annotations,
)

fig.show()

In [376]:
line_data=hourON.groupby(["HourOfTimestamp"]).mean()
line_data.reset_index()

,HourOfTimestamp,MonthOfTimestamp,Mean_consumption,Mean_production,Proportion_of_consumption_production
0,0,6.5,1897.300358,21.152229,0.013947
1,1,6.5,1956.765281,17.956702,0.011225
2,2,6.5,1865.832297,18.072337,0.012273
3,3,6.5,1564.609445,17.979664,0.015867
4,4,6.5,1367.789097,18.410224,0.019663
5,5,6.5,1459.641605,18.505087,0.019570
6,6,6.5,1366.533690,18.427904,0.019125
7,7,6.5,1437.989447,18.418321,0.018445
8,8,6.5,1526.510675,18.363935,0.017956
9,9,6.5,1476.604128,18.353705,0.017658


In [395]:
fig = go.Figure(data=[
    go.Line(name='Mean_consumption', y=(line_data["Mean_consumption"])),
    go.Line(name='Mean_production', y=line_data["Mean_production"]),
    go.Line(name='Mean_energy_balance', y=(line_data["Mean_production"]-line_data["Mean_consumption"]))
]
)

fig.update_layout(margin=dict(
    l=30,
    r=30,
    b=30,
    t=50,
), xaxis_nticks=30,
    yaxis_nticks=20
    # annotations=annotations,
)

fig.show()

/Users/christiannielsen/Documents/Conda_environments/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [392]:
fig = go.Figure(data=[
    go.Line(name='Mean_consumption', y=(line_data["Mean_production"]-line_data["Mean_consumption"])),
    # go.Line(name='Mean_production', y=line_data["Mean_production"])
]
)

fig.update_layout(margin=dict(
    l=30,
    r=30,
    b=30,
    t=50,
), xaxis_nticks=30,
    yaxis_nticks=20
    # annotations=annotations,
)

fig.show()

/Users/christiannielsen/Documents/Conda_environments/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


